In [104]:
import numpy as np
from astropy.io import ascii
from scipy.optimize import root_scalar
from astropy import units as u
from astropy import constants as const
from astropy.time import Time
import matplotlib.pyplot as plt

# Simbolos y constantes 
"""
a = semieje mayor
e = excentricidad
omega = longitud del nodo ascendente
w = argumento del pericentro
i = inclinacion de la orbita
n = movimiento medio
lo = anomalia media en la epoca
tp = tiempo del perihelio
to = tiempo de la epoca
M = la suma de las masas de los cuerpos
"""


# Efemerides Tierra
a_tierra = (1.495582533630905E+8*u.km).to('au')
ecc_tierra = 1.694863932474438E-02
omega_tierra = 1.498625963929686E+02*u.deg
w_tierra = 3.146587763491455E+02*u.deg
i_tierra = 4.164075603038432E-03*u.deg 
n_tierra = 1.141204629731537E-05*u.deg/u.s
l0_tierra = 1.817846947871890E+02*u.deg

#Transformacion de unidades
i_tierra = i_tierra.to('rad')
omega_tierra = omega_tierra.to('rad')
w_tierra = w_tierra.to('rad')
n_tierra = n_tierra.to('rad/s')
l0_tierra = l0_tierra.to('rad')


# Ecuacion de kepler
def Kepler(E, l, ecc):
    f = E - ecc*np.sin(E) - l
    fprime = 1 - ecc*np.cos(E)
    return f, fprime


def iota(t, l0, n, t0,):
    return (l0 + n*(t-t0))


# Orbita de la tierra
t0 = Time('2023-07-09T00:00:00', scale='tdb', format = 'isot')
#to = to.utc

# Provisional
t = Time('2023-07-10T00:00:00', scale='tdb', format = 'isot')

# Hallar iota
iota_tierra = iota(t, l0_tierra, n_tierra, t0)


# Solucionar la Ec. de Kepler
## E_raiz_tierra = root_scalar(Kepler, args=(iota_tierra.value, ecc_tierra), bracket=[0, iota_tierra.value], method="brentq")
E_raiz_tierra = root_scalar(Kepler, args=(iota_tierra.value, ecc_tierra), fprime = True, x0 = 0)



